# PREVISÃO DE RISCOS DE DIABETES COM IA

## **data_processing**

In [2]:
import pandas as pd
import numpy as np

def load_data(filepath):
    """Carrega o dataset a partir do caminho informado."""
    return pd.read_csv(filepath)

def clean_data(df):
    """Substitui valores zero por NaN nas colunas onde zero não faz sentido
    e preenche os valores ausentes com a média da coluna."""
    cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    df[cols_with_zero] = df[cols_with_zero].replace(0, np.nan)
    df.fillna(df.mean(), inplace=True)
    return df

if __name__ == '__main__':
    # Executa carregamento e limpeza
    df = load_data('https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/diabetes.csv')
    df = clean_data(df)

    # Exibe estatísticas do dataframe limpo
    print("Resumo estatístico dos dados tratados:")
    print(df.describe())

Resumo estatístico dos dados tratados:
       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  121.686763      72.405184      29.153420  155.548223   
std       3.369578   30.435949      12.096346       8.790942   85.021108   
min       0.000000   44.000000      24.000000       7.000000   14.000000   
25%       1.000000   99.750000      64.000000      25.000000  121.500000   
50%       3.000000  117.000000      72.202592      29.153420  155.548223   
75%       6.000000  140.250000      80.000000      32.000000  155.548223   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age     Outcome  
count  768.000000                768.000000  768.000000  768.000000  
mean    32.457464                  0.471876   33.240885    0.348958  
std      6.875151                  0.331329   11.760232    0.47695

## **Análise Exploratória**

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

# Configura o visual dos gráficos
sns.set(style = 'whitegrid')

# Cria uma pasta chamada "graficos"
os.makedirs("graficos", exist_ok = True)

# Carrega e trata os dados
def load_and_clean_data(filepath):
    df = pd.read_csv(filepath)
    cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    df[cols_with_zero] = df[cols_with_zero].replace(0, pd.NA)
    df = df.fillna(df.mean())
    return df

# Gera histogramas
def gerarHistogramas(df):
    for col in df.columns[:-1]:
        plt.figure(figsize = (8, 4))
        sns.histplot(data = df, x = col, hue = 'Outcome', kde = True, element = 'step')
        plt.title(f'Histograma de {col}')
        plt.tight_layout()
        plt.savefig(f'graficos/hist_{col}.png')
        plt.close()

# Gera boxplots
def gerarBoxplots(df):
    for col in df.columns[:-1]:
        plt.figure(figsize = (8, 4))
        sns.boxplot(x = 'Outcome', y = col, data = df)
        plt.title(f'Boxplot de {col} por classe')
        plt.tight_layout()
        plt.savefig(f'graficos/box_{col}.png')
        plt.close()

# Mapa de correlação
def gerarHeatmap(df):
        plt.figure(figsize = (10, 8))
        sns.heatmap(df.corr(), annot = True, cmap = 'coolwarm', fmt = ".2f")
        plt.title('Mapa de Correlação')
        plt.tight_layout()
        plt.savefig('graficos/heatmap_correlacao.png')
        plt.close()

# Pairplot
def gerarPairplot(df):
    pair = sns.pairplot(df, hue = 'Outcome', corner = True)
    pair.fig.suptitle('Pairplot das variáveis', y = 1.02)
    pair.savefig('graficos/pairplot.png')
    plt.close()

# Carrega, limpa e gera os gráficos
if __name__ == '__main__':
    caminho_csv = 'https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/diabetes.csv'
    df = load_and_clean_data(caminho_csv)

    gerarHistogramas(df)
    gerarBoxplots(df)
    gerarHeatmap(df)
    gerarPairplot(df)

<ipython-input-4-4102957396>:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(df.mean())


## **Modelagem**

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, classification_report

# Função para dividir o dataframe em treino e teste
def split_data(df, target_col='Outcome', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])  # Features
    y = df[target_col]  # Rótulo (alvo)
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Função para instanciar o modelo de acordo com o nome informado
def get_model(model_name):
    if model_name == 'decision_tree':
        return DecisionTreeClassifier(random_state=42)
    elif model_name == 'random_forest':
        return RandomForestClassifier(n_estimators=100, random_state=42)
    elif model_name == 'logistic_regression':
        return LogisticRegression(max_iter=1000, random_state=42)
    else:
        raise ValueError(f"Modelo '{model_name}' não implementado.")

# Função principal que treina e avalia o modelo
def train_and_evaluate(model_name, filepath='https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/diabetes.csv'):
    raw_df = load_data(filepath)            # Carregamento do CSV bruto
    cleaned_df = clean_data(raw_df)         # Limpeza/preparação dos dados

    X_train, X_test, y_train, y_test = split_data(cleaned_df)  # Separação dos dados

    model = get_model(model_name)           # Seleciona e instancia o modelo
    model.fit(X_train, y_train)             # Treina o modelo

    y_pred = model.predict(X_test)          # Faz previsões no conjunto de teste

    # Breve avaliação do modelo
    print(f"\n--- Resultados para o Modelo: {model_name.replace('_', ' ').title()} ---")
    print("Acurácia:", accuracy_score(y_test, y_pred))
    print("Recall:", recall_score(y_test, y_pred))
    print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

    return model, y_pred, y_test

# Executa o código abaixo apenas se o script for rodado diretamente
if __name__ == "__main__":
    print("Iniciando treinamento e avaliação dos modelos...")

    # Lista com os modelos que vamos testar
    models_to_test = ['decision_tree', 'random_forest', 'logistic_regression']

    trained_models = {}  # Dicionário para guardar os modelos e seus resultados

    for model_name in models_to_test:
        model, y_pred, y_test_actual = train_and_evaluate(model_name)
        trained_models[model_name] = {
            'model': model,
            'y_pred': y_pred,
            'y_test': y_test_actual
        }

Iniciando treinamento e avaliação dos modelos...

--- Resultados para o Modelo: Decision Tree ---
Acurácia: 0.7207792207792207
Recall: 0.6181818181818182
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.79      0.78      0.78        99
           1       0.61      0.62      0.61        55

    accuracy                           0.72       154
   macro avg       0.70      0.70      0.70       154
weighted avg       0.72      0.72      0.72       154


--- Resultados para o Modelo: Random Forest ---
Acurácia: 0.7532467532467533
Recall: 0.6545454545454545
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.81      0.81      0.81        99
           1       0.65      0.65      0.65        55

    accuracy                           0.75       154
   macro avg       0.73      0.73      0.73       154
weighted avg       0.75      0.75      0.75       154


--- Resultados para o Modelo

## **Otimização e Avaliação**

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    recall_score, precision_score, classification_report,
    confusion_matrix
)

def load_data(filepath):
    "Carrega o dataset a partir do caminho informado."
    return pd.read_csv(filepath)

def clean_data(df):
    """Substitui valores zero por NaN nas colunas onde zero não faz sentido
    e preenche os valores ausentes com a mediana da coluna."""
    cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    df[cols_with_zero] = df[cols_with_zero].replace(0, np.nan)
    df.fillna(df.median(), inplace=True)
    return df

def train_model(X_train, y_train):
    "Treina o modelo Random Forest com peso maior para classe positiva."
    model = RandomForestClassifier(class_weight={0: 1, 1: 3}, random_state=42)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test, limiar=0.3):
    "Avalia o modelo com limiar ajustado e imprime métricas."
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= limiar).astype(int)

    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"Recall da classe 1 (positiva): {recall:.4f}")
    print(f"Precision da classe 1 (positiva): {precision:.4f}\n")
    print("Relatório completo:\n")
    print(classification_report(y_test, y_pred, digits=4))

    # Matriz de confusão
    print(f"Matriz de Confusão:\n {cm}")

    return y_pred

if __name__ == '__main__':
    # Caminho do dataset
    filepath = 'https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/diabetes.csv'

    # 1. Carregar e limpar os dados
    df = load_data(filepath)
    df = clean_data(df)

    # 2. Separar variáveis
    X = df.drop("Outcome", axis=1)
    y = df["Outcome"]

    # 3. Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 4. Treinar o modelo
    model = train_model(X_train, y_train)

    # 5. Avaliar o modelo com limiar ajustado
    y_pred = evaluate_model(model, X_test, y_test, limiar=0.3)

Recall da classe 1 (positiva): 0.8519
Precision da classe 1 (positiva): 0.6053

Relatório completo:

              precision    recall  f1-score   support

           0     0.8974    0.7000    0.7865       100
           1     0.6053    0.8519    0.7077        54

    accuracy                         0.7532       154
   macro avg     0.7513    0.7759    0.7471       154
weighted avg     0.7950    0.7532    0.7589       154

Matriz de Confusão:
 [[70 30]
 [ 8 46]]


## **Predição**

In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    recall_score, precision_score, classification_report,
    confusion_matrix
)

def load_data(filepath):
    return pd.read_csv(filepath)

def clean_data(df):
    cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    df[cols_with_zero] = df[cols_with_zero].replace(0, np.nan)
    df.fillna(df.median(), inplace=True)
    return df

def train_model(X_train, y_train):
    model = RandomForestClassifier(class_weight={0: 1, 1: 3}, random_state=42)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test, limiar=0.3):
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= limiar).astype(int)

    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f"Recall da classe 1 (positiva): {recall:.4f}")
    print(f"Precision da classe 1 (positiva): {precision:.4f}\n")
    print("Relatório completo:\n")
    print(classification_report(y_test, y_pred, digits=4))
    print(f"Matriz de Confusão:\n {cm}")

    return y_pred

def predict_patient(model, X_columns):
    print("\nInsira os dados do paciente:")
    entrada = []
    for coluna in X_columns:
        valor = float(input(f"{coluna}: "))
        entrada.append(valor)

    dados_paciente = pd.DataFrame([entrada], columns=X_columns)

    # Prever a probabilidade
    prob = model.predict_proba(dados_paciente)[0][1]
    risco = "ALTO" if prob >= 0.3 else "BAIXO"

    print(f"\nProbabilidade de diabetes: {prob:.2f}")
    print(f"Classificação de risco: {risco}")

if __name__ == '__main__':
    filepath = 'https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/diabetes.csv'

    # 1. Carregar e limpar os dados
    df = load_data(filepath)
    df = clean_data(df)

    # 2. Separar variáveis
    X = df.drop("Outcome", axis=1)
    y = df["Outcome"]

    # 3. Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 4. Treinar o modelo
    model = train_model(X_train, y_train)

    # 5. Avaliar o modelo
    y_pred = evaluate_model(model, X_test, y_test, limiar=0.3)

    # 6. Fazer predição com dados inseridos manualmente
    predict_patient(model, X.columns.tolist())


Recall da classe 1 (positiva): 0.8519
Precision da classe 1 (positiva): 0.6053

Relatório completo:

              precision    recall  f1-score   support

           0     0.8974    0.7000    0.7865       100
           1     0.6053    0.8519    0.7077        54

    accuracy                         0.7532       154
   macro avg     0.7513    0.7759    0.7471       154
weighted avg     0.7950    0.7532    0.7589       154

Matriz de Confusão:
 [[70 30]
 [ 8 46]]

Insira os dados do paciente:
Pregnancies: 1
Glucose: 20
BloodPressure: 10
SkinThickness: 4
Insulin: 14
BMI: 50
DiabetesPedigreeFunction: 1
Age: 50

Probabilidade de diabetes: 0.16
Classificação de risco: BAIXO
